In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from bs4 import Tag
import re
import wikipediaapi as wiki
import time

def remove_footnote(text):
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\[note \d+\]', '', text)
    text = re.sub(r'\xa0', ' ', text)
    text = text.split('\n')
    if len(text)==1:
        return text[0]
    return text

def get_content_list(soup):
    vcard = soup.find("table", {"class" : "infobox vcard"})
    keys = vcard.find_all("th", {"class": "infobox-label"})
    values = vcard.find_all("td", {"class": "infobox-data"})
    keys_lst = [remove_footnote(key.text) for key in keys]
    values_lst = [remove_footnote(value.text) for value in values]
    content_dict = {key: value for key, value in list(zip(keys_lst, values_lst))}
    return content_dict

In [2]:
school_lst = [
    "University of British Columbia",
]

wiki_browser = wiki.Wikipedia('COLX 523 App', 'en')
for school in school_lst:
    page = wiki_browser.page(school)
    if not page.exists():
        continue
    
    url = page.fullurl
    summary = page.summary

    soup = BeautifulSoup(urlopen(url), 'html.parser')
    content_dict = get_content_list(soup)
    content_dict['summary'] = summary

    time.sleep(1)